In [5]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import shutil
import random
import tensorflow as tf 

In [6]:
# peaking into the datset
for dirpath, dirnames, filenames in os.walk("cloud_type"):
    print(f"there are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

there are 3 directories and 0 images in 'cloud_type'.
there are 0 directories and 3698 images in 'cloud_type\test_images_1050'.
there are 0 directories and 5546 images in 'cloud_type\train_images_1050'.
there are 0 directories and 11836 images in 'cloud_type\train_masks_1050'.


In [7]:
# set the directories for the train and test data
train_dir = 'train'
test_dir = 'test'

# create the train and test directories
if not os.path.exists(train_dir):
    os.makedirs(train_dir)

if not os.path.exists(test_dir):
    os.makedirs(test_dir)

In [8]:
mask_dir="cloud_type/train_masks_1050/"
temp_train_img="cloud_type/train_images_1050/"
output_processed_images_dir_Flower="processed_images/Flower/"
output_processed_images_dir_Fish="processed_images/Fish/"
output_processed_images_dir_Gravel="processed_images/Gravel/"
output_processed_images_dir_Sugar="processed_images/Sugar/"

# iterating through the folder for further operations
for mask_name in os.listdir(mask_dir):
    
    #filtering fish type masks
    if "Fish" in mask_name:
        actual_img_name=mask_name[4:]
    #filtering Flower type masks
    elif "Flower" in mask_name:
        actual_img_name=mask_name[6:]

    #filtering Gravel type masks
    elif "Gravel" in mask_name:
        actual_img_name=mask_name[6:]

    #filtering Sugar type masks   
    elif "Sugar" in mask_name:
        actual_img_name=mask_name[5:]
    
    for img_name in os.listdir(temp_train_img):
        if img_name==actual_img_name:
            print(actual_img_name)
        
            #load image and the mask accordingly 
            mask = cv2.imread(os.path.join(mask_dir,mask_name), cv2.IMREAD_GRAYSCALE)
            image= cv2.imread(os.path.join(temp_train_img,img_name))

            # Convert the RGB image to grayscale
            gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            # Apply the mask to the grayscale image
            masked_img = cv2.bitwise_and(gray_img, gray_img, mask=mask)

            # Find the contours of the mask to determine the region of interest
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            x, y, w, h = cv2.boundingRect(contours[0])
        
            # Crop the image according to the region of interest
            cropped_img = masked_img[y:y+h, x:x+w]
        
            # Reshape the image at a desired ratio (e.g. 1:1)
            resized_img = cv2.resize(cropped_img, (256, 256))
        
            #filtering fish type masks
            if "Fish" in mask_name:
                cv2.imwrite(os.path.join(output_processed_images_dir_Fish,mask_name), resized_img)
    
            #filtering Flower type masks
            elif "Flower" in mask_name:
                cv2.imwrite(os.path.join(output_processed_images_dir_Flower,mask_name), resized_img)

            #filtering Gravel type masks
            elif "Gravel" in mask_name:
                cv2.imwrite(os.path.join(output_processed_images_dir_Gravel,mask_name), resized_img)

            #filtering Sugar type masks   
            elif "Sugar" in mask_name:
                cv2.imwrite(os.path.join(output_processed_images_dir_Sugar,mask_name), resized_img)

0011165.jpg
002be4f.jpg
0031ae9.jpg
003994e.jpg
006bf7c.jpg
006c5a6.jpg
008a5ff.jpg
009e2f3.jpg
00bea06.jpg
00dec6a.jpg
0100a84.jpg
0107838.jpg
011ba04.jpg
01242d7.jpg
014310a.jpg
0146ef3.jpg
0153a8b.jpg
015aa06.jpg
015b764.jpg
016c135.jpg
017ded1.jpg
01eecc1.jpg
021bb26.jpg
0225687.jpg
023accd.jpg
0269e9a.jpg
0277e9a.jpg
029b861.jpg
02b6a5d.jpg
02c3e33.jpg
030ac75.jpg
031bfc5.jpg
0326622.jpg
0330f49.jpg
034a383.jpg
034ef1c.jpg
0358d2e.jpg
03a5e35.jpg
03c3906.jpg
03eb81a.jpg
042bba1.jpg
0432314.jpg
043e76c.jpg
043fe61.jpg
0448942.jpg
0451ee4.jpg
045e367.jpg
0488540.jpg
049585f.jpg
04a4276.jpg
04c2acd.jpg
04d5fe8.jpg
04d92ec.jpg
04f9039.jpg
04fe809.jpg
0505b52.jpg
050937d.jpg
054c1e5.jpg
0556b64.jpg
05776fc.jpg
059fc10.jpg
05a1aa8.jpg
05c0dfb.jpg
05e20ba.jpg
05f5370.jpg
060ce39.jpg
0624fa5.jpg
065623d.jpg
0686242.jpg
068771e.jpg
0697495.jpg
0698236.jpg
06b7be8.jpg
06b84fd.jpg
06e5dd6.jpg
06f1f1f.jpg
0721695.jpg
0766057.jpg
0770f53.jpg
07c5a0d.jpg
07c5fc9.jpg
07f403d.jpg
080c004.jpg
0855

In [9]:

#moving unusable images to diffrent dir 
dump_dir="dump/"

# Define threshold for mean pixel value
THRESHOLD = 75

# Define path to dataset
DATASET_PATH = 'processed_images/'

# Define path to dump folder
DUMP_PATH = 'dump/'

# Loop over each image in the dataset
for dirs in os.listdir(DATASET_PATH):
    
    #saving path for future refernce
    rel_path=os.path.join(DATASET_PATH,dirs)
    
    #iterating into subfolder
    for images in os.listdir(rel_path):
        
        full_path=os.path.join(rel_path,images)
        # checking if the selected file not a dir
        if os.path.isfile(full_path):

            # Get the size of the file in bytes and convert to kilobytes
            size_kb = os.path.getsize(full_path) / 1024

            # If the size is below 10kb, move the image to the dump folder
            if size_kb < 10:
                print(f'Moving {images} to dump folder due to low mean pixel value')
                shutil.move(os.path.join(rel_path, images), os.path.join(DUMP_PATH, images))
                continue
    

Moving Fish002be4f.jpg to dump folder due to low mean pixel value
Moving Fish003994e.jpg to dump folder due to low mean pixel value
Moving Fish006bf7c.jpg to dump folder due to low mean pixel value
Moving Fish008a5ff.jpg to dump folder due to low mean pixel value
Moving Fish009e2f3.jpg to dump folder due to low mean pixel value
Moving Fish00bea06.jpg to dump folder due to low mean pixel value
Moving Fish0100a84.jpg to dump folder due to low mean pixel value
Moving Fish0107838.jpg to dump folder due to low mean pixel value
Moving Fish01242d7.jpg to dump folder due to low mean pixel value
Moving Fish014310a.jpg to dump folder due to low mean pixel value
Moving Fish0153a8b.jpg to dump folder due to low mean pixel value
Moving Fish015b764.jpg to dump folder due to low mean pixel value
Moving Fish0225687.jpg to dump folder due to low mean pixel value
Moving Fish0269e9a.jpg to dump folder due to low mean pixel value
Moving Fish02c3e33.jpg to dump folder due to low mean pixel value
Moving Fis

In [10]:


# set the percentage of images to use for testing
test_pct = 0.2

# set the root directory for the image data
data_root = 'processed_images'

# set the directories for the train and test data
train_dir = 'train'
test_dir = 'test'

# create the train and test directories
if not os.path.exists(train_dir):
    os.makedirs(train_dir)

if not os.path.exists(test_dir):
    os.makedirs(test_dir)
    
# iterate over each class directory
for class_dir in os.listdir(data_root):
    class_path = os.path.join(data_root, class_dir)

    # skip any non-directory files
    if not os.path.isdir(class_path):
        continue

    print(f"Processing images in directory: {class_path}")

    # create the subdirectories for this class in the train and test directories
    train_class_dir = os.path.join(train_dir, class_dir)
    test_class_dir = os.path.join(test_dir, class_dir)
    os.makedirs(train_class_dir, exist_ok=True)
    os.makedirs(test_class_dir, exist_ok=True)

    # get the list of image files in the class directory
    image_files = [os.path.join(class_path, filename) for filename in os.listdir(class_path)
                   if os.path.isfile(os.path.join(class_path, filename))]

    # shuffle the list of image files
    random.shuffle(image_files)

    # split the image files into train and test sets based on the split percentage
    split_idx = int(len(image_files) * (1 - test_pct))
    train_images = image_files[:split_idx]
    test_images = image_files[split_idx:]

    # copy the train images to the train directory
    for image_path in train_images:
        image_name = os.path.basename(image_path)
        shutil.copy2(image_path, os.path.join(train_class_dir, image_name))

    # copy the test images to the test directory
    for image_path in test_images:
        image_name = os.path.basename(image_path)
        shutil.copy2(image_path, os.path.join(test_class_dir, image_name))

print("Done!")

Processing images in directory: processed_images\Fish
Processing images in directory: processed_images\Flower
Processing images in directory: processed_images\Gravel
Processing images in directory: processed_images\Sugar
Done!


In [11]:
IMG_SIZE = 224
TRAIN_DIR = 'train'
TEST_DIR = 'test'
CLASS_LABELS = os.listdir(TRAIN_DIR)

In [12]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(    rescale=1./255,
                                                                    validation_split=0.2    )

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                                                                    rescale=1./255      )


In [13]:
batch_size=64
train_generator = train_datagen.flow_from_directory(    TRAIN_DIR,
                                                        target_size=(224, 224),
                                                        batch_size=batch_size,
                                                        class_mode ='categorical',
                                                        subset='training',
                                                        shuffle=True,
                                                        classes=CLASS_LABELS    )

validation_generator = train_datagen.flow_from_directory(   TRAIN_DIR,
                                                            target_size=(224, 224),
                                                            batch_size=batch_size,
                                                            class_mode='categorical',
                                                            subset='validation',
                                                            shuffle=True,
                                                            classes=CLASS_LABELS    )

test_generator = test_datagen.flow_from_directory(  TEST_DIR,
                                                    target_size=(224, 224),
                                                    batch_size=1,
                                                    class_mode='categorical',
                                                    shuffle =False,
                                                    classes=CLASS_LABELS    )

Found 4369 images belonging to 4 classes.
Found 1090 images belonging to 4 classes.
Found 1366 images belonging to 4 classes.


In [14]:
tf.keras.backend.clear_session()

In [15]:
from tensorflow.keras import layers, models
model = models.Sequential()

# Add convolutional layers
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(224,224,3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Flatten the output from convolutional layers and add dense layers
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(4, activation='softmax'))


model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 64)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 128)     73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 256)       295168    
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 26, 26, 256)      0

In [16]:
history=model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples//batch_size,
    epochs=6,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples//batch_size
)

Epoch 1/6
68/68 [==============================] - 63s 496ms/step - loss: 1.4040 - accuracy: 0.3456 - val_loss: 1.2208 - val_accuracy: 0.5055
Epoch 2/6
68/68 [==============================] - 14s 208ms/step - loss: 1.0704 - accuracy: 0.5110 - val_loss: 0.8991 - val_accuracy: 0.6103
Epoch 3/6
68/68 [==============================] - 15s 214ms/step - loss: 0.9497 - accuracy: 0.5805 - val_loss: 0.9216 - val_accuracy: 0.5772
Epoch 4/6
68/68 [==============================] - 15s 216ms/step - loss: 0.9043 - accuracy: 0.6074 - val_loss: 0.8412 - val_accuracy: 0.6360
Epoch 5/6
68/68 [==============================] - 14s 207ms/step - loss: 0.8903 - accuracy: 0.6107 - val_loss: 0.9045 - val_accuracy: 0.6112
Epoch 6/6
68/68 [==============================] - 14s 207ms/step - loss: 0.8597 - accuracy: 0.6307 - val_loss: 0.8943 - val_accuracy: 0.6287


In [17]:
from sklearn.metrics import classification_report

# Load the test data
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        'test',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        shuffle=False)

# Predict the classes for the test data
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

# Generate the classification report
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

report = classification_report(true_classes, predicted_classes, target_names=class_labels)

print(report)


Found 1366 images belonging to 4 classes.
43/43 [==============================] - 11s 221ms/step
              precision    recall  f1-score   support

        Fish       0.56      0.56      0.56       296
      Flower       0.58      0.44      0.50       238
      Gravel       0.56      0.30      0.39       362
       Sugar       0.63      0.93      0.75       470

    accuracy                           0.60      1366
   macro avg       0.58      0.56      0.55      1366
weighted avg       0.59      0.60      0.57      1366



In [18]:
# Define the window size and stride for the sliding window
WINDOW_SIZE = (224, 224)
STRIDE = 224

# Load the satellite image
image = cv2.imread('3DIMG_01APR2022_0730_L1C_ASIA_IR1.jpg')

# Define the coordinates of the top-left and bottom-right corners of the desired crop
x1, y1 = 10, 100
x2, y2 = 1915, 1480

# Crop the image
image = image[y1:y2, x1:x2]

# Define the colors for each class
colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0)]


# Initialize an empty list to store the detected cloud regions
regions = []

# Slide the window across the image and predict the presence of each class in each window
for y in range(0, image.shape[0] - WINDOW_SIZE[0], STRIDE):
    for x in range(0, image.shape[1] - WINDOW_SIZE[1], STRIDE):
        window = image[y:y+WINDOW_SIZE[0], x:x+WINDOW_SIZE[1]]
        window_resized = cv2.resize(window, (224, 224))
        window_rescaled = window_resized / 255.0
        window_expanded = tf.expand_dims(window_rescaled, axis=0)
        predictions = model.predict(window_expanded)[0]
        predicted_class = np.argmax(predictions)
        if predictions[predicted_class] > 0.5:
            regions.append((x, y, x+WINDOW_SIZE[1], y+WINDOW_SIZE[0], predicted_class))

# Draw bounding boxes around the detected regions with different colors for each class
for (x1, y1, x2, y2, predicted_class) in regions:
    cv2.rectangle(image, (x1, y1), (x2, y2), colors[predicted_class], 2)

# Display the output image
output_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
cv2.imwrite('output_image.jpg', output_image)
cv2.imshow('Cloud Detection', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

1/1 [==============================] - 0s 25ms/step
